In [ ]:
!gcloud auth login
!mkdir /content/images/
!gsutil -mq cp -r gs://lily-li-dataset/satire/defaults/images/ /content/images/
!gsutil -mq cp gs://lily-li-dataset/satire/defaults/**.jsonl /content/

In [ ]:
import numpy as np
import jsonlines
from PIL import Image
import tensorflow as tf

with jsonlines.open('/content/train.jsonl', 'r') as data:
    hl_train = []
    img_train = []
    y_train = []
    for d in data:
        try:
            filepath = "/content/images/images/" + str(d["id"]) + ".jpg"  
            img = Image.open(filepath).resize((128,128), Image.BICUBIC).convert('RGB')
            img_train.append(np.asarray(img) / 255.)
            hl_train.append(d["text"])
            y_train.append(d["label"])
        except:
            print("Image at filepath {0} does not exist".format(filepath))

with jsonlines.open('/content/test.jsonl', 'r') as data:
    hl_test = []
    img_test= []
    y_test = []
    for d in data:
        try:
            filepath = "/content/images/images/" + str(d["id"]) + ".jpg"  
            img = Image.open(filepath).resize((128,128), Image.BICUBIC).convert('RGB')
            img_test.append(np.asarray(img) / 255.)
            hl_test.append(d["text"])
            y_test.append(d["label"])
        except:
            print("Image at filepath {0} does not exist".format(filepath))

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [ ]:
img_train = np.stack(img_train, axis=0)
img_test = np.stack(img_test, axis=0)

img_train = tf.convert_to_tensor(img_train)
img_test = tf.convert_to_tensor(img_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

In [ ]:
def create_model():
    ResNet = tf.keras.applications.ResNet101(
            include_top=False,
            weights="imagenet",
            input_shape=(128,128,3),
            )

    input_img = tf.keras.layers.Input(shape=(128,128,3))
    img_output = ResNet(input_img)
    flattened = tf.keras.layers.Flatten()(img_output)

    dense = tf.keras.layers.Dense(256, activation='relu')(flattened)
    dropout = tf.keras.layers.Dropout(0.5)(dense)

    output = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)(dense)

    model = tf.keras.Model(
        inputs=input_img,
        outputs=output,
    )
    return model

In [ ]:
model = create_model()
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
resnet101 (Functional)       (None, 4, 4, 2048)        42658176  
_________________________________________________________________
flatten_2 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               8388864   
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 51,047,297
Trainable params: 50,941,953
Non-trainable params: 105,344
_________________________________________________________________


In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=2e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.binary_crossentropy,
              metrics=["accuracy", "AUC", "Precision", "Recall"])

In [ ]:
model.fit(x=img_train, y=y_train, batch_size=32, epochs=15, validation_steps=2000//32, steps_per_epoch=8000//32,
           validation_data=(img_test, y_test))

Epoch 1/15
250/250 [==============================] - 57s 228ms/step - loss: 0.7163 - accuracy: 0.6609 - auc: 0.6965 - precision: 0.5831 - recall: 0.5378 - val_loss: 1.7813 - val_accuracy: 0.6008 - val_auc: 0.5162 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/15
250/250 [==============================] - 54s 218ms/step - loss: 0.1314 - accuracy: 0.9703 - auc: 0.9954 - precision: 0.9714 - recall: 0.9538 - val_loss: 0.8914 - val_accuracy: 0.5272 - val_auc: 0.5094 - val_precision: 0.4083 - val_recall: 0.4104
Epoch 3/15
250/250 [==============================] - 55s 218ms/step - loss: 0.0409 - accuracy: 0.9960 - auc: 0.9998 - precision: 0.9966 - recall: 0.9934 - val_loss: 0.9846 - val_accuracy: 0.5998 - val_auc: 0.6069 - val_precision: 0.4964 - val_recall: 0.1717
Epoch 4/15
250/250 [==============================] - 55s 218ms/step - loss: 0.0209 - accuracy: 0.9986 - auc: 0.9998 - precision: 0.9988 - recall: 0.9978 - val_loss: 0.7354 - val_accuracy: 0.7132 - val_auc: 0.7672 -